# Summary Tables
This notebook will create summaries of the data with the following format:

![wanted_output](format2.png)

In [182]:
import pandas as pd
import numpy as np
import seaborn as sns
from typing import Tuple, List

rename_dict = {"hilo": "Amos HiLo", "mixed": "Amos Mixed", "tourlousse": "Tourlousse"}
metrics = ["FPRA", "Sens", "AD"]
wanted_pipelines = ["biobakery3", "biobakery4", "jams", "wgsa2", "woltka"]

def initialize_datasets() -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Read in the data
    df = pd.read_csv('../results/all_stats_species.csv')
    df = df.loc[df["threshold"] == 0.0001]
    # If the source is mixed, hilo, or tourlousse, then we need to average the samples together.
    replicate_comm = ["mixed", "hilo", "tourlousse"]

    wanted_cols_one_to_one = metrics + ["Community", "Pipeline"]
    wanted_cols_replicates = metrics + ["Pipeline", "Source"]

    replicates = df.loc[df["Source"].isin(replicate_comm) & df["Pipeline"].isin(wanted_pipelines)]
    one_to_one = df.loc[~df["Source"].isin(replicate_comm) & df["Pipeline"].isin(wanted_pipelines)]

    # Average together replicates within same source and pipeline
    replicates = replicates.groupby(["Pipeline", "Source"]).agg({
        "FPRA": ["count", "mean", "std"],
        "Sens": ["count", "mean", "std"],
        "AD": ["count", "mean", "std"],
    })

    replicates = replicates.groupby(["Source", "Pipeline"]).mean().reset_index()
    replicates = replicates.loc[:, wanted_cols_replicates]

    return replicates, one_to_one

replicates, one_to_one = initialize_datasets()

def make_replicate_df(replicates: pd.DataFrame, metric: str) -> pd.DataFrame:
    test = replicates[["Pipeline", "Source", metric]]
    test = test.droplevel(0, axis=1)
    test.columns = ["Pipeline", "Source", "count", "mean", "std"]
    test.index.name = metric
    test = test.pivot(index="Source", columns="Pipeline", values=["count", "mean", "std"])
    # Rename the sources
    test = test.rename(index=rename_dict)
    # test.to_csv("{}_replicates.csv".format(metric))
    # display(test)

    reorganized = test.groupby(level=1, axis=1).apply(lambda x: x.droplevel(1, axis=1))

    # Rename the columns
    rename_cols = {"count": "n", "mean": "Mean", "std": "Stdev", "Source": "Community"}
    reorganized = reorganized.rename(columns=rename_cols)
    reorganized.index.name = "Community"

    return reorganized



In [183]:
one_to_one_rename = {
    "EG nist": "NIST EG",
    "MIX-A nist": "NIST MIX-A",
    "MIX-B nist": "NIST MIX-B",
    "MIX-C nist": "NIST MIX-C",
    "MIX-D nist": "NIST MIX-D",
    "S1 bmock12": "BMock12",
    "S1 camisimGI": "CamiSim S1",
    "S2 camisimGI": "CamiSim S2",
}
def make_one_to_one_dfs(df: pd.DataFrame) -> pd.DataFrame:
    one_to_one_copy = df.copy()
    one_to_one_copy["Community"] = one_to_one["SampleID"] + " " + one_to_one["Source"]
    # display(one_to_one_copy.head())
    one_to_one_copy = one_to_one_copy[["FPRA", "Sens", "AD", "Community", "Pipeline"]]
    # display(one_to_one_copy.head())

    # display(one_to_one_copy.head())
    one_to_one_copy["n"] = 1
    one_to_one_copy["Stdev"] = np.NaN
    # We want the five pipelines as the columns
    # one_to_one_copy["n"] = 1
    piv_df = one_to_one_copy.pivot(index="Community", columns="Pipeline", values=["FPRA", "Sens", "AD", "n", "Stdev"])
    piv_df = piv_df.rename(index=one_to_one_rename)

    return piv_df


# The current data should be moved into a multiindex called "Mean".
# The current index should be moved into a column called "Pipeline"
def reorganize_df(df: pd.DataFrame) -> pd.DataFrame:
    reorganized = df.groupby(level=1, axis=1).apply(lambda x: x.droplevel(1, axis=1))
    return reorganized


# one_to_one_copy.to_csv("one_to_one.csv", index=True)

In [196]:
piv_df = make_one_to_one_dfs(one_to_one)
for metric in metrics:
    print(metric)
    reorganized_replicate = make_replicate_df(replicates, metric)
    # display(piv_df[["n", metric, "Stdev"]])
    reorganized_one_to_one = reorganize_df(piv_df[["n", metric, "Stdev"]])
    # Sort by alphabetical order
    reorganized_one_to_one = reorganized_one_to_one.sort_index()

    # Rename FPRA, Sens, AD to Mean
    rename_cols = {"FPRA": "Mean", "Sens": "Mean", "AD": "Mean"}
    reorganized_one_to_one = reorganized_one_to_one.rename(columns=rename_cols)

    # display(reorganized_replicate)
    # display(reorganized_one_to_one)
    # Stack the two dataframes together
    stacked = pd.concat([reorganized_one_to_one, reorganized_replicate], axis=0)

    # Drop all columns with n except the first one
    # stacked = stacked.drop(stacked.columns[stacked.columns.get_level_values(1) == "n"][1:], axis=1, level=1)
    stacked = stacked.drop(stacked.columns[stacked.columns.get_level_values(1) == "n"][1:], axis=1)
    display(stacked)

    stacked.to_csv(f"{metric}.csv")

# dfs = [piv_df[["n", i, "Stdev"]] for i in metrics]

# for df in dfs:
#     reorganized = reorganize_df(df)
#     display(reorganized)


FPRA


Pipeline   biobakery3                     biobakery4                 jams  \
                    n      Mean     Stdev       Mean     Stdev       Mean   
Community                                                                   
BMock12           1.0   2.63030       NaN  17.926600       NaN  50.043800   
CamiSim S1        1.0   0.52650       NaN   3.056600       NaN   1.438800   
CamiSim S2        1.0   0.05860       NaN   3.834400       NaN   0.205500   
NIST EG           1.0   2.66450       NaN   0.000000       NaN   0.210400   
NIST MIX-A        1.0   0.84640       NaN   0.000000       NaN   0.052000   
NIST MIX-B        1.0  11.41540       NaN   0.014100       NaN   0.265500   
NIST MIX-C        1.0  32.05670       NaN  32.309400       NaN  22.333500   
NIST MIX-D        1.0   0.06070       NaN   0.102400       NaN   0.019400   
Amos HiLo         5.0   0.00954  0.021332   0.000000  0.000000   3.269400   
Amos Mixed        5.0   3.86412  0.220681   0.000000  0.000000   9.755280   
Tourlousse        6.0  12.22505  0.142976   4.716617  0.037888   9.789317   

Pipeline                  wgsa2               woltka            
               Stdev       Mean     Stdev       Mean     Stdev  
Community                                                       
BMock12          NaN  37.344200       NaN  11.964700       NaN  
CamiSim S1       NaN   2.043200       NaN   8.509100       NaN  
CamiSim S2       NaN   3.910200       NaN  30.666800       NaN  
NIST EG          NaN   1.160100       NaN  26.480200       NaN  
NIST MIX-A       NaN   0.678100       NaN  30.915200       NaN  
NIST MIX-B       NaN   1.086600       NaN  31.056800       NaN  
NIST MIX-C       NaN   9.162000       NaN  69.168200       NaN  
NIST MIX-D       NaN   0.250600       NaN  13.194800       NaN  
Amos HiLo   0.115418   4.065480  0.096760  22.239680  0.205998  
Amos Mixed  1.431971  14.063520  0.115006  27.391800  0.307410  
Tourlousse  0.222829  20.190933  0.093907  26.144283  0.248612

Sens


Pipeline   biobakery3                 biobakery4             jams            \
                    n      Mean Stdev       Mean Stdev       Mean     Stdev   
Community                                                                     
BMock12           1.0   33.3333   NaN    58.3333   NaN   50.00000       NaN   
CamiSim S1        1.0  100.0000   NaN    89.4737   NaN   97.36840       NaN   
CamiSim S2        1.0  100.0000   NaN    95.2381   NaN  100.00000       NaN   
NIST EG           1.0   85.7143   NaN    92.8571   NaN   92.85710       NaN   
NIST MIX-A        1.0   81.8182   NaN    90.9091   NaN   72.72730       NaN   
NIST MIX-B        1.0   72.7273   NaN    72.7273   NaN   81.81820       NaN   
NIST MIX-C        1.0   63.6364   NaN    63.6364   NaN   90.90910       NaN   
NIST MIX-D        1.0   72.7273   NaN    72.7273   NaN   81.81820       NaN   
Amos HiLo         5.0   89.4737   0.0   100.0000   0.0   98.94736  2.353775   
Amos Mixed        5.0   94.7368   0.0   100.0000   0.0  100.00000  0.000000   
Tourlousse        6.0   78.9474   0.0    94.7368   0.0  100.00000  0.000000   

Pipeline       wgsa2         woltka        
                Mean Stdev     Mean Stdev  
Community                                  
BMock12      50.0000   NaN   8.3333   NaN  
CamiSim S1   97.3684   NaN  86.8421   NaN  
CamiSim S2  100.0000   NaN  80.9524   NaN  
NIST EG     100.0000   NaN  85.7143   NaN  
NIST MIX-A  100.0000   NaN  90.9091   NaN  
NIST MIX-B  100.0000   NaN  90.9091   NaN  
NIST MIX-C  100.0000   NaN  81.8182   NaN  
NIST MIX-D  100.0000   NaN  81.8182   NaN  
Amos HiLo    89.4737   0.0  94.7368   0.0  
Amos Mixed   89.4737   0.0  94.7368   0.0  
Tourlousse   89.4737   0.0  94.7368   0.0

AD


Pipeline   biobakery3                      biobakery4                jams  \
                    n       Mean     Stdev       Mean     Stdev      Mean   
Community                                                                   
BMock12           1.0  22.145700       NaN   16.83270       NaN  33.71770   
CamiSim S1        1.0   8.618800       NaN   12.76470       NaN  15.03610   
CamiSim S2        1.0   1.230800       NaN    7.11380       NaN   4.85250   
NIST EG           1.0  10.182600       NaN    8.94460       NaN  12.25310   
NIST MIX-A        1.0   9.585200       NaN    2.84260       NaN  12.35270   
NIST MIX-B        1.0  13.652200       NaN    9.81400       NaN  13.55040   
NIST MIX-C        1.0  14.108700       NaN   11.56900       NaN  17.16010   
NIST MIX-D        1.0   4.134500       NaN    4.10980       NaN  11.21660   
Amos HiLo         5.0   8.402520  1.069797    2.71252  0.040068  12.79564   
Amos Mixed        5.0   8.767440  0.774772    1.99916  0.055893  10.42296   
Tourlousse        6.0  14.567183  0.725752    3.80855  0.024836   8.16020   

Pipeline                  wgsa2               woltka            
               Stdev       Mean     Stdev       Mean     Stdev  
Community                                                       
BMock12          NaN  44.160000       NaN  42.140700       NaN  
CamiSim S1       NaN  13.476400       NaN  30.759600       NaN  
CamiSim S2       NaN   6.378600       NaN  25.876600       NaN  
NIST EG          NaN  10.663800       NaN  22.595300       NaN  
NIST MIX-A       NaN   6.997500       NaN  19.242000       NaN  
NIST MIX-B       NaN  13.079200       NaN  22.875100       NaN  
NIST MIX-C       NaN  14.702000       NaN  24.102600       NaN  
NIST MIX-D       NaN   8.931600       NaN  21.115500       NaN  
Amos HiLo   0.341404  12.436960  0.286740  16.384500  0.112320  
Amos Mixed  0.358741  14.815740  0.051594  20.629920  0.216708  
Tourlousse  0.396388  14.378967  0.100234  23.903417  0.260645